# Iteración de Valores en Frozen Lake

## Introducción

La iteración de valores es un algoritmo de programación dinámica utilizado en problemas de decisión secuenciales para encontrar la política óptima que maximiza la recompensa acumulada a largo plazo. Este método se basa en la ecuación de Bellman, que descompone el problema en subproblemas más pequeños, facilitando la búsqueda de la solución óptima.

### Ecuación de Bellman

La ecuación de Bellman para el valor de un estado \( V(s) \) en un problema de decisión secuencial se define como:

$$
[ V(s) = \max_a \sum_{s'} P(s'|s,a) \left[ R(s, a, s') + \gamma V(s') \right] ]
$$

Donde:
- $( V(s) )$: Valor del estado $( s )$.

- $( a )$: Acción.

- $( P(s'|s,a) )$: Probabilidad de transitar al estado $( s' )$ desde el estado $( s )$ tomando la acción $( a )$.

- $( R(s, a, s') )$: Recompensa inmediata recibida después de transitar del estado $( s )$ al estado $( s' )$ tomando la acción $( a )$.

- $( \gamma )$: Factor de descuento, que determina la importancia de las recompensas futuras.

### Diferencias con la Ecuación Simplificada

Anteriormente utilizamos una versión simplificada de la ecuación de Bellman para entornos determinísticos:

$$[ V(s) = \max_a \left[ R(s,a) + \gamma V(s') \right] ]$$

En la versión simplificada, asumimos que la transición entre estados es determinística, es decir, cada acción lleva a un estado siguiente fijo. Sin embargo, en el caso de la iteración de valores en un entorno estocástico, consideramos la probabilidad de transitar a cada estado $( s' )$ dado un estado actual $( s )$ y una acción $( a )$. Esto se refleja en la ecuación más general donde sumamos sobre todos los posibles estados siguientes $( s' )$ ponderados por su probabilidad $( P(s'|s,a) )$.


### Algoritmo de Iteración de Valores

El algoritmo de iteración de valores utiliza la ecuación de Bellman para actualizar iterativamente los valores de los estados hasta que convergen a los valores óptimos. Luego, se deriva la política óptima seleccionando la acción que maximiza el valor esperado para cada estado.


### Determinístico vs Estocástico

Es importante decidir si el entorno debe ser determinístico o estocástico:

- **Entorno Determinístico**: Cada acción produce siempre el mismo resultado predecible. Esto puede ser útil para una primera implementación sencilla.

- **Entorno Estocástico**: Las acciones pueden tener resultados probabilísticos, lo cual añade realismo y complejidad al problema. Utilizar `is_slippery=True` en Frozen Lake introduce esta característica, haciendo que las acciones no siempre resulten en los movimientos esperados, lo que puede reflejar mejor las incertidumbres del mundo real.

### 1. Instalación de Librerías

Primero, necesitamos instalar las librerías necesarias. Esto incluye `gymnasium` para el entorno de Frozen Lake y `ffmpeg` para la grabación de videos.

In [121]:
!pip install gymnasium
!apt-get install -y ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


### 2. Importación de Librerías

Importamos las librerías necesarias para nuestro proyecto.

In [122]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
from gymnasium.wrappers import RecordVideo

## 3. Calculando la Función de Valor Óptima

Definiremos una función llamada (`value_iteration`) donde calculamos la función de valor óptima iterativamente tomando el máximo sobre la función Q.

La función de iteración de valores (`value_iteration`) utiliza la ecuación de Bellman para actualizar los valores de los estados y encontrar la política óptima.

In [123]:
# Establecer la semilla para reproducibilidad
seed = 42
np.random.seed(seed)

In [124]:
def value_iteration(env):
    # Definimos el número de iteraciones
    num_iterations = 20000

    # Definimos el umbral para verificar la convergencia de la función de valor
    threshold = 1e-8

    # Definimos el factor de descuento
    gamma = 0.99

    # Inicializamos la tabla de valores con el valor de todos los estados a cero
    value_table = np.zeros(env.observation_space.n)

    # Para cada iteración
    for i in range(num_iterations):

        # Actualizamos la tabla de valores, es decir, en cada iteración utilizamos la tabla de valores actualizada
        # (valores de estado) de la iteración anterior
        updated_value_table = np.copy(value_table)

        # Calculamos la función de valor (valor del estado) tomando el máximo del valor Q.
        # Para cada estado, calculamos los valores Q de todas las acciones en el estado y luego
        # actualizamos el valor del estado como el que tiene el valor Q máximo, como se muestra a continuación:
        for s in range(env.observation_space.n):

            Q_values = [sum([prob * (reward + gamma * updated_value_table[next_state] * (not terminated))
                             for prob, next_state, reward, terminated in env.P[s][a]])
                        for a in range(env.action_space.n)]

            value_table[s] = max(Q_values)

        # Después de calcular la tabla de valores, es decir, el valor de todos los estados, verificamos si la
        # diferencia entre la tabla de valores obtenida en la iteración actual y la iteración anterior es
        # menor o igual a un valor umbral. Si es menor, rompemos el bucle y devolvemos la
        # tabla de valores como nuestra función de valor óptima, como se muestra a continuación:
        if np.sum(np.fabs(updated_value_table - value_table)) <= threshold:
            print(f"Converged after {i + 1} iterations")
            break

    return value_table

## 4. Extrayendo la Política Óptima de la Función de Valor Óptima

En el paso anterior, calculamos la función de valor óptima. Ahora, veamos cómo extraer la política óptima de la función de valor óptima calculada.

Definimos una función llamada `extract_policy` que toma la `value_table` como parámetro:

In [125]:
def extract_policy(env, value_table):
    # Definimos el factor de descuento
    gamma = 0.99

    # Inicializamos la política con ceros, es decir, primero establecemos las acciones para todos los estados en cero
    policy = np.zeros(env.observation_space.n, dtype=int)

    # Calculamos la función Q usando la función de valor óptima obtenida en el
    # paso anterior. Después de calcular la función Q, podemos extraer la política seleccionando la acción que tiene
    # el valor Q máximo. Dado que estamos calculando la función Q usando la función de valor
    # óptima, la política extraída de la función Q será la política óptima.

    # Para cada estado
    for s in range(env.observation_space.n):

        # Calculamos el valor Q de todas las acciones en el estado
        Q_values = [sum([prob * (reward + gamma * value_table[next_state] * (not terminated))
                         for prob, next_state, reward, terminated in env.P[s][a]])
                    for a in range(env.action_space.n)]

        # Extraemos la política seleccionando la acción que tiene el valor Q máximo
        policy[s] = np.argmax(Q_values)

    return policy

## 5. Visualizando la Política Óptima

La función `visualize_policy` se utiliza para mostrar cómo el agente sigue la política óptima en el entorno.

In [126]:
def visualize_policy(env, policy):
    state = env.reset()[0]
    terminated = False
    truncated = False
    while not terminated and not truncated:
        action = policy[state]
        state, reward, terminated, truncated, _ = env.step(action)

## 6. Ejecutando la Iteración de Valores y Grabando el Video

La función `run_value_iteration_and_record` ejecuta el algoritmo de iteración de valores, aplica la política óptima y graba un video del desempeño del agente.


In [127]:
def run_value_iteration_and_record(episodes=1, video_path='frozen_lake_video', render=False, num_iterations=10000):
    env = gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=True, render_mode='rgb_array')
    env = RecordVideo(env, video_path, episode_trigger=lambda episode_id: True)
    value_table = value_iteration(env)
    policy = extract_policy(env, value_table)

    print("Policy:")
    print(policy.reshape((4, 4)))
    print("Value Function:")
    print(value_table.reshape((4, 4)))

    for _ in range(episodes):
        visualize_policy(env, policy)

    # Analizar transiciones
    state = 0  # Estado inicial (0,0)
    for action in range(env.action_space.n):
        print(f"Action {action} from state {state}:")
        for prob, next_state, reward, done in env.P[state][action]:
            print(f"  -> Prob: {prob}, Next State: {next_state}, Reward: {reward}, Done: {done}")

    env.close()

In [128]:
run_value_iteration_and_record(episodes=1, video_path='frozen_lake_video')

Converged after 488 iterations
Policy:
[[0 3 3 3]
 [0 0 0 0]
 [3 1 0 0]
 [0 2 1 0]]
Value Function:
[[0.5420259  0.49880315 0.47069564 0.45685165]
 [0.55845093 0.         0.35834805 0.        ]
 [0.59179872 0.64307981 0.61520754 0.        ]
 [0.         0.74172043 0.86283742 0.        ]]
Moviepy - Building video /content/frozen_lake_video/rl-video-episode-0.mp4.
Moviepy - Writing video /content/frozen_lake_video/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /content/frozen_lake_video/rl-video-episode-0.mp4
Action 0 from state 0:
  -> Prob: 0.3333333333333333, Next State: 0, Reward: 0.0, Done: False
  -> Prob: 0.3333333333333333, Next State: 0, Reward: 0.0, Done: False
  -> Prob: 0.3333333333333333, Next State: 4, Reward: 0.0, Done: False
Action 1 from state 0:
  -> Prob: 0.3333333333333333, Next State: 0, Reward: 0.0, Done: False
  -> Prob: 0.3333333333333333, Next State: 4, Reward: 0.0, Done: False
  -> Prob: 0.3333333333333333, Next State: 1, Reward: 0.0, Done: False
Action 2 from state 0:
  -> Prob: 0.3333333333333333, Next State: 4, Reward: 0.0, Done: False
  -> Prob: 0.3333333333333333, Next State: 1, Reward: 0.0, Done: False
  -> Prob: 0.3333333333333333, Next State: 0, Reward: 0.0, Done: False
Action 3 from state 0:
  -> Prob: 0.3333333333333333, Next State: 1, Reward: 0.0, Done: False
  -> Prob: 0.3333333333333333, Next State: 0, Reward: 0.0, Done: False
  -> Prob: 0.3333333333333333, Next St

#### Entorno Estocástico

En un entorno estocástico, las acciones tienen resultados probabilísticos. La política debe tener en cuenta esta incertidumbre y proporcionar acciones que maximicen la probabilidad de alcanzar el objetivo, incluso con resultados no deterministas. Esto puede resultar en una política que parece contraintuitiva, pero que está optimizada para las condiciones del entorno.

### Análisis de la Política en el Estado Inicial (0,0)

#### Transiciones en el Estado Inicial con Acción 0 (Izquierda)

```python
Action 0 from state 0:
  -> Prob: 0.3333333333333333, Next State: 0, Reward: 0.0, Done: False
  -> Prob: 0.3333333333333333, Next State: 0, Reward: 0.0, Done: False
  -> Prob: 0.3333333333333333, Next State: 4, Reward: 0.0, Done: False
```

- **Acción 0 (Izquierda)**: Tiene una alta probabilidad de mantener al agente en el mismo estado (0,0) y una probabilidad de 1/3 de moverse al estado (4,0), que es seguro.
- Esta acción minimiza el riesgo de caer en un hoyo cercano en las primeras etapas del juego.

### 7. Reproducción del Video en Google Colab

In [129]:
from IPython.display import HTML
from base64 import b64encode

video_file = open("frozen_lake_video/rl-video-episode-0.mp4", "rb").read()
video_url = "data:video/mp4;base64," + b64encode(video_file).decode()
HTML(f"<video width=400 controls><source src='{video_url}' type='video/mp4'></video>")

### Explicación de la política
#### Política en Entorno Determinístico

```python
Policy:
[[1 2 1 0]
 [1 0 1 0]
 [2 1 1 0]
 [0 2 2 0]]
```

- La política sigue una ruta clara y directa hacia el objetivo.
- Cada acción lleva al siguiente estado predecible.

#### Política en Entorno Estocástico

```python
Policy:
[[0 3 3 3]
 [0 0 0 0]
 [3 1 0 0]
 [0 2 1 0]]
```

- La política considera las probabilidades de deslizamiento y toma acciones que maximizan la recompensa esperada.
- Las acciones pueden parecer subóptimas a primera vista, pero están diseñadas para maximizar la expectativa considerando la probabilidad de deslizarse y terminar en estados con mayores valores esperados.
